In [ ]:
!pip install xgboost
!pip install imblearn


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler,label_binarize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score, f1_score,precision_score, recall_score, roc_auc_score, average_precision_score,roc_curve, precision_recall_curve, hamming_loss,auc
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import mutual_info_classif
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
data=pd.read_csv("/content/drive/MyDrive/Dataset_SRIP/reduced_dataset.csv")

In [20]:
data

,frame.time,frame.len,frame.protocols,eth.src,eth.dst,ip.dst,ip.src,ip.flags,ip.ttl,ip.proto,...,tcp.dstport,tcp.flags,tcp.window_size_value,tcp.window_size_scalefactor,tcp.checksum,tcp.options,tcp.pdu.size,udp.srcport,udp.dstport,label
0,"Jan 14, 2025 18:40:22.447710000 GMT",67,eth:ethertype:ip:udp:dns,02:42:52:d7:fa:00,0c:6e:9c:16:00:00,192.168.0.2,192.168.18.17,0x02,64,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48322.0,53.0,Benign
1,"Jan 14, 2025 18:40:22.453402000 GMT",83,eth:ethertype:ip:udp:dns,0c:6e:9c:16:00:00,02:42:52:d7:fa:00,192.168.18.17,192.168.0.2,0x02,61,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.0,48322.0,Benign
2,"Jan 14, 2025 18:40:22.453507000 GMT",90,eth:ethertype:ip:udp:ntp,02:42:52:d7:fa:00,0c:6e:9c:16:00:00,192.168.0.3,192.168.18.17,0x02,64,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46343.0,123.0,Benign
3,"Jan 14, 2025 18:40:22.458119000 GMT",90,eth:ethertype:ip:udp:ntp,0c:6e:9c:16:00:00,02:42:52:d7:fa:00,192.168.18.17,192.168.0.3,0x02,61,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123.0,46343.0,Benign
4,"Jan 14, 2025 18:40:22.560013000 GMT",76,eth:ethertype:ip:udp:dns,02:42:52:d7:fa:00,0c:6e:9c:16:00:00,192.168.0.2,192.168.18.17,0x02,64,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36848.0,53.0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141155,"Jan 14, 2025 23:48:13.342081000 GMT",54,eth:ethertype:ip:tcp,02:42:f7:f5:f7:00,0c:6e:9c:16:00:00,192.168.35.10,192.168.18.15,0x02,64,6,...,58481.0,0x0014,0.0,-2.0,0xd7a6,NaN,NaN,NaN,NaN,Unknown
141156,"Jan 14, 2025 23:48:13.362412000 GMT",58,eth:ethertype:ip:tcp,0c:6e:9c:16:00:00,02:42:f7:f5:f7:00,192.168.18.15,192.168.35.10,0x00,251,6,...,8061.0,0x0002,1025.0,NaN,0xf12c,020405b4,NaN,NaN,NaN,Unknown
141157,"Jan 14, 2025 23:48:13.362441000 GMT",54,eth:ethertype:ip:tcp,02:42:f7:f5:f7:00,0c:6e:9c:16:00:00,192.168.35.10,192.168.18.15,0x02,64,6,...,58481.0,0x0014,0.0,-2.0,0x0cd7,NaN,NaN,NaN,NaN,Unknown
141158,"Jan 14, 2025 23:48:13.387536000 GMT",58,eth:ethertype:ip:tcp,0c:6e:9c:16:00:00,02:42:f7:f5:f7:00,192.168.18.15,192.168.35.10,0x00,251,6,...,8084.0,0x0004,1200.0,-2.0,0xbb4a,020405b4,NaN,NaN,NaN,Unknown


In [ ]:
folder_path = "/content/data/GothamDataset2025/*.csv"

data = dd.read_csv(folder_path, dtype=str)


In [21]:
data["label"].value_counts()

,count
label,
Benign,10000
Ingress Tool Transfer,10000
TCP Scan,10000
Telnet Brute Force,10000
Mirai UDP Flooding,10000
Merlin TCP Flooding,10000
Merlin ICMP Flooding,10000
Merlin C&C Communication,10000
Mirai GRE Flooding,10000


In [ ]:
data.head()

,frame.time,frame.len,frame.protocols,eth.src,eth.dst,ip.dst,ip.src,ip.flags,ip.ttl,ip.proto,...,tcp.dstport,tcp.flags,tcp.window_size_value,tcp.window_size_scalefactor,tcp.checksum,tcp.options,tcp.pdu.size,udp.srcport,udp.dstport,label
0,"Jan 14, 2025 18:40:22.447710000 GMT",67,eth:ethertype:ip:udp:dns,02:42:52:d7:fa:00,0c:6e:9c:16:00:00,192.168.0.2,192.168.18.17,0x02,64,17,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,48322.0,53.0,Benign
1,"Jan 14, 2025 18:40:22.453402000 GMT",83,eth:ethertype:ip:udp:dns,0c:6e:9c:16:00:00,02:42:52:d7:fa:00,192.168.18.17,192.168.0.2,0x02,61,17,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,53.0,48322.0,Benign
2,"Jan 14, 2025 18:40:22.453507000 GMT",90,eth:ethertype:ip:udp:ntp,02:42:52:d7:fa:00,0c:6e:9c:16:00:00,192.168.0.3,192.168.18.17,0x02,64,17,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,46343.0,123.0,Benign
3,"Jan 14, 2025 18:40:22.458119000 GMT",90,eth:ethertype:ip:udp:ntp,0c:6e:9c:16:00:00,02:42:52:d7:fa:00,192.168.18.17,192.168.0.3,0x02,61,17,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,123.0,46343.0,Benign
4,"Jan 14, 2025 18:40:22.560013000 GMT",76,eth:ethertype:ip:udp:dns,02:42:52:d7:fa:00,0c:6e:9c:16:00:00,192.168.0.2,192.168.18.17,0x02,64,17,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,36848.0,53.0,Benign


In [ ]:
label_counts = data["label"].value_counts().compute()

print(label_counts)


label
Mirai UDP Flooding           8897895
Merlin ICMP Flooding           57580
File Download                   7196
UDP Scan                        4242
Benign                      12256883
Telnet Brute Force            227649
Mirai GRE Flooding           5911401
Merlin UDP Flooding            29996
CoAP Amplification            274837
Merlin TCP Flooding           120000
TCP Scan                      737764
Merlin C&C Communication       29356
Ingress Tool Transfer          21587
C&C Communication                528
Reporting                        450
Mirai TCP Flooding           6548173
Unknown                         7670
Mirai C&C Communication         1074
Name: count, dtype: int64[pyarrow]


In [ ]:
labels = data['label'].unique().compute()

print("Found labels:", labels)

# Process each label separately
for label in labels:
    print(f"Processing label = {label}")
    # Select rows of that label and take first 10k
    sample = data[data['label'] == label].head(10000, compute=True)

    # Save to disk immediately, don't keep in RAM
    sample.to_csv(f"/content/sample_{label}.csv", index=False)

print("Done! One file per label saved.")


Found labels: 0          Mirai UDP Flooding
0        Merlin ICMP Flooding
0               File Download
0                    UDP Scan
0                      Benign
0          Telnet Brute Force
0          Mirai GRE Flooding
0         Merlin UDP Flooding
0          CoAP Amplification
0         Merlin TCP Flooding
0                    TCP Scan
0    Merlin C&C Communication
0       Ingress Tool Transfer
0           C&C Communication
0                   Reporting
0          Mirai TCP Flooding
0                     Unknown
0     Mirai C&C Communication
Name: label, dtype: string
Processing label = Mirai UDP Flooding


/usr/local/lib/python3.12/dist-packages/dask/dataframe/core.py:382: UserWarning: Insufficient elements for `head`. 10000 elements requested, only 0 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


Processing label = Merlin ICMP Flooding


/usr/local/lib/python3.12/dist-packages/dask/dataframe/core.py:382: UserWarning: Insufficient elements for `head`. 10000 elements requested, only 0 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


Processing label = File Download


...

/usr/local/lib/python3.12/dist-packages/dask/dataframe/core.py:382: UserWarning: Insufficient elements for `head`. 10000 elements requested, only 0 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


Processing label = Unknown


/usr/local/lib/python3.12/dist-packages/dask/dataframe/core.py:382: UserWarning: Insufficient elements for `head`. 10000 elements requested, only 857 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


Processing label = Mirai C&C Communication
Done! One file per label saved.


/usr/local/lib/python3.12/dist-packages/dask/dataframe/core.py:382: UserWarning: Insufficient elements for `head`. 10000 elements requested, only 0 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


In [ ]:
folder_path = "/content/"

# Collect all per-label sample CSVs
sample_files = glob.glob(os.path.join(folder_path, "sample_*.csv"))

print("Merging files:", sample_files)


merged_df = pd.concat([pd.read_csv(f) for f in sample_files], ignore_index=True)

# Save the merged file
merged_df.to_csv("/content/merged_samples.csv", index=False)

Merging files: ['/content/sample_Mirai GRE Flooding.csv', '/content/sample_Mirai C&C Communication.csv', '/content/sample_UDP Scan.csv', '/content/sample_Reporting.csv', '/content/sample_Merlin TCP Flooding.csv', '/content/sample_Merlin ICMP Flooding.csv', '/content/sample_Telnet Brute Force.csv', '/content/sample_Ingress Tool Transfer.csv', '/content/sample_Unknown.csv', '/content/sample_Mirai UDP Flooding.csv', '/content/sample_Merlin C&C Communication.csv', '/content/sample_C&C Communication.csv', '/content/sample_Merlin UDP Flooding.csv', '/content/sample_Benign.csv', '/content/sample_TCP Scan.csv', '/content/sample_CoAP Amplification.csv', '/content/sample_File Download.csv', '/content/sample_Mirai TCP Flooding.csv']


/tmp/ipython-input-3814198186.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat([pd.read_csv(f) for f in sample_files], ignore_index=True)


In [ ]:
merged_df

,frame.time,frame.len,frame.protocols,eth.src,eth.dst,ip.dst,ip.src,ip.flags,ip.ttl,ip.proto,...,tcp.dstport,tcp.flags,tcp.window_size_value,tcp.window_size_scalefactor,tcp.checksum,tcp.options,tcp.pdu.size,udp.srcport,udp.dstport,label
0,"Jan 18, 2025 21:29:53.490203000 GMT",54,eth:ethertype:ip:tcp,0c:6e:9c:16:00:00,02:42:52:d7:fa:00,192.168.18.17,192.168.0.100,0x00,61,6,...,2323.0,0x0002,26161.0,NaN,0x4d3e,NaN,NaN,NaN,NaN,Telnet Brute Force
1,"Jan 18, 2025 21:29:53.490308000 GMT",54,eth:ethertype:ip:tcp,02:42:52:d7:fa:00,0c:6e:9c:16:00:00,192.168.0.100,192.168.18.17,0x02,64,6,...,36064.0,0x0014,0.0,-2.0,0xb35c,NaN,NaN,NaN,NaN,Telnet Brute Force
2,"Jan 18, 2025 21:30:07.171884000 GMT",54,eth:ethertype:ip:tcp,0c:6e:9c:16:00:00,02:42:52:d7:fa:00,192.168.18.17,192.168.0.100,0x00,61,6,...,2323.0,0x0002,26161.0,NaN,0x4d3e,NaN,NaN,NaN,NaN,Telnet Brute Force
3,"Jan 18, 2025 21:30:07.171948000 GMT",54,eth:ethertype:ip:tcp,02:42:52:d7:fa:00,0c:6e:9c:16:00:00,192.168.0.100,192.168.18.17,0x02,64,6,...,36064.0,0x0014,0.0,-2.0,0xb35c,NaN,NaN,NaN,NaN,Telnet Brute Force
4,"Jan 18, 2025 21:30:18.685523000 GMT",54,eth:ethertype:ip:tcp,0c:6e:9c:16:00:00,02:42:52:d7:fa:00,192.168.18.17,192.168.0.100,0x00,61,6,...,23.0,0x0002,26161.0,NaN,0x563a,NaN,NaN,NaN,NaN,Telnet Brute Force
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28194,"Jan 18, 2025 21:38:48.530375000 GMT",58,eth:ethertype:ip:tcp,02:42:52:d7:fa:00,0c:6e:9c:16:00:00,192.168.20.10,192.168.18.17,0x02,64,6,...,26914.0,0x0012,64240.0,NaN,0x857c,020405b4,NaN,NaN,NaN,TCP Scan
28195,"Jan 18, 2025 21:38:48.539945000 GMT",54,eth:ethertype:ip:tcp,0c:6e:9c:16:00:00,02:42:52:d7:fa:00,192.168.18.17,192.168.20.10,0x02,62,6,...,23.0,0x0004,0.0,-2.0,0xcc80,NaN,NaN,NaN,NaN,TCP Scan
28196,"Jan 18, 2025 21:38:48.542981000 GMT",74,eth:ethertype:ip:tcp,0c:6e:9c:16:00:00,02:42:52:d7:fa:00,192.168.18.17,192.168.20.10,0x02,62,6,...,23.0,0x0002,64240.0,NaN,0x2781,020405b40402080a7eb4935e0000000001030307,NaN,NaN,NaN,TCP Scan
28197,"Jan 18, 2025 21:38:48.543058000 GMT",74,eth:ethertype:ip:tcp,02:42:52:d7:fa:00,0c:6e:9c:16:00:00,192.168.20.10,192.168.18.17,0x02,64,6,...,47768.0,0x0012,65160.0,NaN,0x6efe,020405b40402080a08b45fd97eb4935e01030307,NaN,NaN,NaN,TCP Scan


In [ ]:
merged_df['label'].value_counts()

,count
label,
Telnet Brute Force,10000
TCP Scan,10000
Benign,7233
Unknown,857
Ingress Tool Transfer,109


In [ ]:
data=data.drop_duplicates()

In [ ]:
data=data.drop(columns=["pkSeqID","stime","flgs","proto","saddr","sport","daddr","dport","state","ltime","seq"])

In [ ]:
data.isnull().sum()

In [ ]:
data.head()

ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+--------------+--------+----------+
| Column       | Found  | Expected |
+--------------+--------+----------+
| tcp.checksum | object | float64  |
| tcp.flags    | object | float64  |
| tcp.options  | object | float64  |
+--------------+--------+----------+

The following columns also raised exceptions on conversion:

- tcp.checksum
  ValueError("could not convert string to float: '0xc4fa'")
- tcp.flags
  ValueError("could not convert string to float: '0x0002'")
- tcp.options
  ValueError("could not convert string to float: '020405b40402080ac4c967b60000000001030307'")

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'tcp.checksum': 'object',
       'tcp.flags': 'object',
       'tcp.options': 'object'}

to the call to `read_csv`/`read_table`.